In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
from sklearn import preprocessing
from sklearn import linear_model
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR, LinearSVR
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import GridSearchCV

In [3]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
ruta_archivo = '/content/drive/MyDrive/CS Datos Grupo 9/telco_churn_clusterai.csv'
customers_df = pd.read_csv(ruta_archivo)

In [ ]:
filas, columnas = customers_df.shape
print(f"Filas: {filas}; Columnas: {columnas}")

In [ ]:
customers_df.head(5)

In [ ]:
#borro la columna que es un indice, tambien "customerID" ya que es un campo indintificatorio por cliente tambien y no tendra relevancia
customers_df =customers_df.drop(columns=['Unnamed: 0', 'customerID'])

In [ ]:
customers_df.info()

In [ ]:
#remplazo los espacios en blanco por nulos para poder identificarlos
customers_df.replace(' ', np.nan, inplace=True)
#vemos los valores nulos por columna
print("\nValores nulos por columna:")
customers_df.isnull().sum()

In [ ]:
# Calculamos % los valores faltantes para cada una de las variables
miss = pd.DataFrame(customers_df.isnull().mean(), columns=["Missing"])
perc_miss = miss.loc[miss.Missing > 0]
perc_miss.sort_values("Missing", ascending = False)

In [ ]:
#no borramos filas porque es mucha data que se pierde, entonces hacemos un tratamientop sobre los nulos, dependiendo que represente cada campo


In [ ]:
print("\nEstadasticas:")
print(customers_df.describe(include='all'))

**texto en negrita**###

In [ ]:
# Armo una lista de las columnas del tipo objeto o con "categorias", que tengan valores nulos
categorical_cols = customers_df.select_dtypes(include=['object']).isnull().any().loc[lambda x: x].index.tolist()
print(categorical_cols)

In [ ]:
#a las columnas categorias, le imputamos la moda a los nulos
for col in categorical_cols:
    if col in customers_df.columns:
        # Imputar con la moda
        customers_df[col].fillna(customers_df[col].mode()[0], inplace=True)

In [ ]:
# Armo una lista de las columnas del tipo float o numericas, que tengan valores nulos
numeric_cols= customers_df.select_dtypes(include=['float64']).isnull().any().loc[lambda x: x].index.tolist()
print(numeric_cols)


In [ ]:
#a las columnas numericoas, le imputamos la media
for col in numeric_cols:
    if col in customers_df.columns:
        customers_df[col].fillna(customers_df[col].median(), inplace=True) # la mediana es menos sensible a outliers

In [ ]:
# verifico si quedan nulos
print("Valores nulos restantes luego de las imputaciones:")
print(customers_df.isnull().sum())

In [ ]:
print("\nEstadisticas luego tratamiento:")
print(customers_df.describe(include='all'))

# Seleccion y trabajo de features con los que vamos a trabajar

In [ ]:
# Evaluo los valores de las columnas; observo que hay columnas donde se podria remplazar el "No tiene servicio" por un simple "No"

for col in customers_df.columns:
    unique_values = customers_df[col].unique()
    print(f"{col}: {unique_values}")

In [ ]:
# Cambio el tipo de dato de la columna "TotalCharges" ya que esta como objeto pero es float
customers_df['TotalCharges'] = customers_df['TotalCharges'].astype(float)

customers_df['TotalCharges'].dtype

In [ ]:
# Para ello armo una lista de las columnas con esta clase de datos en el dataframe "customers_df"
columns_with_no_service = []

for column in customers_df.columns:
    unique_values = customers_df[column].unique()
    if ('No internet service' in unique_values) or ('No phone service' in unique_values):
        columns_with_no_service.append(column)

print(columns_with_no_service)

In [ ]:
# Remplazo el "No internet service" por un "No" en la lista de las columnas buscadas
for column in columns_with_no_service:
   customers_df[column] = customers_df[column].replace(['No internet service','No phone service'], 'No')

In [ ]:
# Verifico que haya funcionado de la manera correcta en una de las columnas
customers_df['MultipleLines'].unique()

In [ ]:
# Paso los valores de categoricos a numericos. Creando dummies para aquellos con mas de 2 valores posibles y para los que tiene menos de 2 valores simplemente lo factorizo

categorical_cols = customers_df.select_dtypes(include=['object']).columns

for col in categorical_cols:
    if customers_df[col].nunique() > 2:
        dummies = pd.get_dummies(customers_df[col], prefix=col, drop_first=False, dtype= float)
        customers_df = pd.concat([customers_df, dummies], axis=1)
        customers_df = customers_df.drop(columns=[col])
    else:
        customers_df[col] = pd.factorize(customers_df[col])[0]


In [ ]:
# Verifico el dataframe transformado
customers_df.head()

### TRATAMIENTO DE OUTLIERS

In [ ]:
# Seleccionar columnas numéricas continuas
numerical_cols = ['tenure','MonthlyCharges','TotalCharges']

# Mostrar estadísticas descriptivas para las variables numéricas
print("Estadísticas descriptivas de las variables numéricas continuas:")
print(customers_df[numerical_cols].describe())

# Detectar y visualizar outliers con boxplots
for col in numerical_cols:
    plt.figure(figsize=(10, 5))
    sns.boxplot(x=customers_df[col], palette="Set2")
    plt.title(f"Boxplot para {col}")
    plt.xlabel(col)
    plt.show()

    # Mostrar histograma para ver la distribución
    plt.figure(figsize=(10, 5))
    sns.histplot(customers_df[col], kde=True, color="blue", bins=30)
    plt.title(f"Histograma para {col}")
    plt.xlabel(col)
    plt.ylabel("Frecuencia")
    plt.show()

# Identificar valores atípicos según el rango intercuartílico (IQR)
def detectar_outliers(customers_df, columnas):
    outliers = {}
    for col in columnas:
        Q1 = customers_df[col].quantile(0.25)  # Primer cuartil
        Q3 = customers_df[col].quantile(0.75)  # Tercer cuartil
        IQR = Q3 - Q1                # Rango intercuartílico
        lim_inferior = Q1 - 1.5 * IQR
        lim_superior = Q3 + 1.5 * IQR

        # Detectar filas que están fuera del rango
        outliers[col] = customers_df[(customers_df[col] < lim_inferior) | (customers_df[col] > lim_superior)][col]

        print(f"\nOutliers detectados en {col}:")
        print(f"  Límite inferior: {lim_inferior:.2f}, Límite superior: {lim_superior:.2f}")
        print(f"  Total de outliers: {len(outliers[col])}")

    return outliers

# Detectar outliers
outliers_detectados = detectar_outliers(customers_df, numerical_cols)

# NO ENCONTRAMOS OUTLIERS DENTRO DE LOS CAMPOS NUMERICOS `CONTINUOS`

In [ ]:
#GENERAMOS LA MATRIZ DE CORRELACIONES
corr_matrix = customers_df.corr()
corr_matrix_redondeada = corr_matrix.round(1)
#print(corr_matrix_redondeada)
plt.figure(figsize=(25, 20))
sns.heatmap(corr_matrix_redondeada, annot=True)
plt.title("Matriz de Correlacion (Redondeada a 1 Decimal)")
plt.show()

In [ ]:
# No se encuentra una correlacion entre las features, eseptuando a las dummies creadas por feature, las cuales siguen un correlacion entre ellas

### BUSCAMOS RELACION ENTRE LAS FEATURES Y CHURN

In [ ]:
# Variables numéricas
numerical_cols = ['tenure', 'MonthlyCharges', 'TotalCharges']

x = customers_df.copy() # Creo una copia para no modificar el DF original

# Boxplots para comparar distribuciones
for col in numerical_cols:
    plt.figure(figsize=(8, 5))
    sns.boxplot(x=customers_df['Churn'], y=customers_df[col], palette="Set2")
    plt.title(f"Boxplot de {col} vs Churn")
    plt.show()

# Densidad (KDE) para ver diferencias en distribuciones
for col in numerical_cols:
    plt.figure(figsize=(8, 5))
    sns.kdeplot(data=customers_df, x=col, hue='Churn', fill=True)
    plt.title(f"Distribución de {col} por Churn")
    plt.show()

### Los clientes con mayor permanencia(tenure) y mayor gasto total(TotalCharges) tienden a ser menos propensos a abandonar, mientras que los cargos mensuales mas altos podrian estar asociados con una mayor probabilidad de abandono.
### Tambien hay un churn temprano, con una mediana para clientes con tenure entre 10 y 20

In [ ]:
dummy_cols = [
    'InternetService_DSL', 'InternetService_Fiber optic', 'InternetService_No',
    'Contract_Month-to-month', 'Contract_One year', 'Contract_Two year',
    'PaymentMethod_Bank transfer (automatic)', 'PaymentMethod_Credit card (automatic)',
    'PaymentMethod_Electronic check', 'PaymentMethod_Mailed check'
]

for col in dummy_cols:
    plt.figure(figsize=(8, 5))

    # Calcular la proporcion de Churn para cada categoria de las variables dummy
    prop_churn = customers_df.groupby(col)['Churn'].value_counts(normalize=True).unstack()

    # Graficar proporciones
    prop_churn.plot(kind='bar', stacked=True, colormap='coolwarm', alpha=0.8)
    plt.title(f"Distribución de Churn para {col}")
    plt.xlabel(col)
    plt.ylabel("Proporción")
    plt.legend(title="Churn")
    plt.xticks(ticks=[0, 1], labels=['0 (No)', '1 (Yes)'], rotation=0)
    plt.tight_layout()
    plt.show()

### VEMOS QUE PARA LOS CAMPOS:
InternetService_Fiber optic = 1
Contract_Month-to-month = 1
PaymentMethod_Electronic check = 1
### APROXIMADAMENTE EL 40% EN CADA UNO SE GENERA CHURN
Vamos a verificar si simultaneamente hay alguna relacion

In [ ]:
# Filtrar clientes que cumplen las tres condiciones
filtered_customers = customers_df[
    (customers_df['InternetService_Fiber optic'] == 1) &
    (customers_df['Contract_Month-to-month'] == 1) &
    (customers_df['PaymentMethod_Electronic check'] == 1)
]

# Calcular el total de clientes en este grupo
total_filtered = filtered_customers.shape[0]

# Calcular cuántos de estos tienen Churn = Yes
churn_filtered = filtered_customers[filtered_customers['Churn'] == 'Yes'].shape[0]

# Porcentaje de Churn en este grupo
churn_percentage = (churn_filtered / total_filtered) * 100 if total_filtered > 0 else 0

# Mostrar resultados
print(f"Total de clientes que cumplen las condiciones: {total_filtered}")
print(f"Clientes con Churn = Yes: {churn_filtered}")
print(f"Porcentaje de Churn en este grupo: {churn_percentage:.2f}%")

El resultado indica que ninguno de los clientes que tienen estas características simultaneamente ha abandonado el servicio (Churn = Yes). Esto  sugiere que esta combinacion podría estar asociada con una alta retencion de clientes.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics, svm, datasets
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, roc_curve, auc,confusion_matrix, ConfusionMatrixDisplay

In [ ]:
X = customers_df.drop('Churn', axis=1)  #todas las columnas excepto 'Churn'
y = customers_df['Churn']  # Objetivo: 'Churn'

In [ ]:
# dividimos los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [ ]:
# Step 4: auto scaling train- set

scaler = StandardScaler()
scaler.fit(X_train)

In [ ]:
# Escalado de variables

X_train_scal = scaler.fit_transform(X_train)
X_test_scal = scaler.transform(X_test)


In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(16,5))
sns.heatmap(X, ax = axs[0])
sns.heatmap(X_train_scal, ax = axs[1])
sns.heatmap(X_test_scal, ax = axs[2])
axs[0].set_title('Data pre-standarization')
axs[1].set_title('Standarized train data')
axs[2].set_title('Standarized test data')

In [ ]:
# Creo el grid search con unos hiperparametros arbitrarios. Además hago 5 crossvalidation
param_grid = {'C': [0.1,1,5 ]}
clf_LR = GridSearchCV(LogisticRegression(), param_grid, cv=5, verbose=1, n_jobs=-1)

In [ ]:
# fiteamos el dataset con el modelo
clf_LR.fit(X_train_scal, y_train)


In [ ]:
# Vemos cual fue el mejor parametro para C
print("The best parameter is %s with a score of %0.2f" % (clf_LR.best_params_, clf_LR.best_score_))


In [ ]:
#hacemos las predicciones
y_pred_LR = clf_LR.predict(X_test_scal)  # Predecir en los datos de prueba

In [ ]:
#evaluamos el modelo
accuracy = accuracy_score(y_test, y_pred_LR)  # calcula la precision
print(f"Precisión: {accuracy}")
print(classification_report(y_test, y_pred_LR))  # muestra el informe de clasificacion

In [ ]:
#compute and plot AUC
y_proba = clf_LR.predict_proba(X_test_scal)
fpr1, tpr1, thresholds = roc_curve(y_test.astype('int'), y_proba[:,1], drop_intermediate = False)
auc_LR = metrics.auc(fpr1, tpr1)
print("El AUC con Regression Lineal  es = " + str(auc_LR))

In [ ]:
# Crear la figura y los subplots (1 fila, 2 columnas)
fig, axes = plt.subplots(1, 2, figsize=(12, 6))  # 1 fila, 2 columnas

# Gráfico 1: ROC Curve
axes[0].plot(fpr1, tpr1, lw=2, alpha=0.7, label='ROC Curve', color='b')
axes[0].plot([0,1], [0,1], linestyle='--', lw=1, color='r', label='Luck', alpha=.8)
axes[0].set_xlabel('Falsos Positivos Rate')
axes[0].set_ylabel('Verdaderos Positivos Rate')
axes[0].grid(False)
axes[0].legend(loc="lower right")
axes[0].set_title('ROC con LR ')

# Gráfico 2: Matriz de Confusión
cm = confusion_matrix(y_test, y_pred_LR)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf_LR.best_estimator_.classes_)
disp.plot(cmap='Blues', ax=axes[1])  # Usar el segundo eje para el gráfico
axes[1].set_title("Matriz de Confusión LR")

plt.show()


# PCA Logistic Regression

In [ ]:
# cantidad de componentes a extraer
n_comps = 10
# definimos PCA
pca = PCA(n_components= n_comps)
# fit_transform del PCA a nuestros datos
xpca_LR = pca.fit_transform(X_train_scal)
xpca_test_LR = pca.fit_transform(X_test_scal)
# obtenemos los auto-valores
eigenvalues_LR = pca.explained_variance_ratio_

In [ ]:
sns.heatmap(xpca_LR)

In [ ]:
# Graficamos
components = range(1,n_comps + 1)

eigenvalues_acum = pca.explained_variance_ratio_.cumsum()

# Eje Izquierdo
fig, ax1 = plt.subplots(figsize=(12,6))
ax1.set_xlabel('Componentes Principales',fontsize=20)
ax1.set_ylabel('Varianza Explicada', color='k',fontsize=20)
ax1.bar(components, eigenvalues_LR, color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# Eje derecho
ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
ax2.set_ylabel('Varianza Acumulada', color='k',fontsize=20)
ax2.plot(components, eigenvalues_acum, color='red')
ax2.tick_params(axis='y', labelcolor='red')

fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()

In [ ]:
# Visualizo con las dos variables con mas varianza
plt.figure(figsize=(9,6))
plt.scatter((pd.DataFrame(xpca_LR)).iloc[:,0],(pd.DataFrame(xpca_LR)).iloc[:,1])
plt.xlabel('PC 1')
plt.ylabel('PC 2')
plt.title('Figura de PC1 y PC2')
plt.show()

In [ ]:
#entrenamos y evaluamos el modelo con PCA
clf_LR_pca = LogisticRegression(C = clf_LR.best_params_['C'])
clf_LR_pca.fit(xpca_LR, y_train)# Ajusta el modelo con PCA a los datos de entrenamiento
y_pred_pca = clf_LR_pca.predict(xpca_test_LR)  # Predece en los datos de prueba
accuracy_pca = accuracy_score(y_test, y_pred_pca)  # Calcula la precision con PCA
print(f"Precision con PCA: {accuracy_pca}")
print(classification_report(y_test, y_pred_pca))  # Mostrar informe de clasificacion con PCA

In [ ]:
y_proba_pca = clf_LR_pca.predict_proba(xpca_test_LR)
fpr2, tpr2, thresholds = roc_curve(y_test.astype('int'), y_proba_pca[:,1], drop_intermediate = False)
auc_LR = metrics.auc(fpr2, tpr2)
print("El AUC con Regression Lineal  es = " + str(auc_LR))

In [ ]:
# Crear la figura y los subplots (1 fila, 2 columnas)
fig, axes = plt.subplots(1, 2, figsize=(12, 6))  # 1 fila, 2 columnas

# Gráfico 1: ROC Curve
axes[0].plot(fpr2, tpr2, lw=2, alpha=0.7, label='ROC Curve', color='b')
axes[0].plot(["No se fueron", "Se fueron"], ["No se fueron", "Se fueron"], linestyle='--', lw=1, color='r', label='Luck', alpha=.8)
axes[0].set_xlabel('Falsos Positivos Rate')
axes[0].set_ylabel('Verdaderos Positivos Rate')
axes[0].grid(False)
axes[0].legend(loc="lower right")
axes[0].set_title('ROC con LR y PCA')

# Gráfico 2: Matriz de Confusión
cm2 = confusion_matrix(y_test, y_pred_pca)
disp = ConfusionMatrixDisplay(confusion_matrix=cm2, display_labels=clf_LR_pca.classes_)
disp.plot(cmap='Blues', ax=axes[1])  # Usar el segundo eje para el gráfico
axes[1].set_title("Matriz de Confusión LR Y PCA")

plt.show()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
#Hacemos un Pipeline con RF
pipeline_rf = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', RandomForestClassifier(random_state=42))
])

In [ ]:
#Entrenamos y evaluamos con RF
pipeline_rf.fit(X_train, y_train)
y_pred_rf = pipeline_rf.predict(X_test)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f"Precisión con Random Forest: {accuracy_rf}")
print(classification_report(y_test, y_pred_rf))

In [ ]:
#PCA para RF
pipeline_pca_rf = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=0.95)),
    ('classifier', RandomForestClassifier(random_state=42))
])

In [ ]:
#Entrenamos y evaluamos el RF con PCA
pipeline_pca_rf.fit(X_train, y_train)
y_pred_pca_rf = pipeline_pca_rf.predict(X_test)
accuracy_pca_rf = accuracy_score(y_test, y_pred_pca_rf)
print(f"Precisión con Random Forest y PCA: {accuracy_pca_rf}")
print(classification_report(y_test, y_pred_pca_rf))

# Super Vector Machine

In [ ]:
# Hyperparametros
parameters = {'kernel':('linear', 'rbf'),
              'C':[1, 10, 20, 100],
              'gamma':[0.1,1, 10]}
#
clf_SVM = GridSearchCV(svm.SVC(probability= True), param_grid = parameters, refit = True, cv = 3, verbose=2)

clf_SVM.fit(X_train_scal, y_train)

In [ ]:
print("The best parameters are %s with a score of %0.2f" % (clf_SVM.best_params_, clf_SVM.best_score_))


In [ ]:
#Evaluamos
y_pred_SVM = clf_SVM.predict(X_test_scal)
accuracy_svm = accuracy_score(y_test, y_pred_SVM)
print(f"Precision con SVM: {accuracy_svm}")
print(classification_report(y_test,y_pred_SVM ))

In [ ]:
y_proba_SVM = clf_SVM.predict_proba(X_test_scal)
fpr3, tpr3, thresholds = roc_curve(y_test.astype('int'), y_proba_SVM[:,1], drop_intermediate = False)
auc_SVM= metrics.auc(fpr3, tpr3)
print("El AUC con SVM  es = " + str(auc_SVM))

In [ ]:
# Crear la figura y los subplots (1 fila, 2 columnas)
fig, axes = plt.subplots(1, 2, figsize=(12, 6))  # 1 fila, 2 columnas

# Gráfico 1: ROC Curve
axes[0].plot(fpr3, tpr3, lw=2, alpha=0.7, label='ROC Curve', color='b')
axes[0].plot(["No se fueron", "Se fueron"], ["No se fueron", "Se fueron"], linestyle='--', lw=1, color='r', label='Luck', alpha=.8)
axes[0].set_xlabel('Falsos Positivos Rate')
axes[0].set_ylabel('Verdaderos Positivos Rate')
axes[0].grid(False)
axes[0].legend(loc="lower right")
axes[0].set_title('ROC con SVM ')

# Gráfico 2: Matriz de Confusión
cm3 = confusion_matrix(y_test, y_pred_SVM)

disp = ConfusionMatrixDisplay(confusion_matrix=cm3, display_labels=clf_SVM.classes_)
disp.plot(cmap='Blues', ax=axes[1])  # Usar el segundo eje para el gráfico
axes[1].set_title("Matriz de Confusión SVM")

plt.show()

In [ ]:

#ahora con PCA
clf_SVM_pca = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=10)),
    ('classifier', svm.SVC(C = clf_SVM.best_params_['C'],kernel = clf_SVM.best_params_['kernel'], gamma = clf_SVM.best_params_['gamma'], probability = True, random_state=42))
])

clf_SVM_pca.fit(X_train, y_train)
y_pred_pca_svm = clf_SVM_pca.predict(X_test)
accuracy_pca_svm = accuracy_score(y_test, y_pred_pca_svm)
print(f"Precision con SVM y PCA: {accuracy_pca_svm}")
print(classification_report(y_test, y_pred_pca_svm))

In [ ]:
y_proba_SVM = clf_SVM_pca.predict_proba(X_test_scal)
fpr4, tpr4, thresholds = roc_curve(y_test.astype('int'), y_proba_SVM[:,1], drop_intermediate = False)
auc_SVM_pca= metrics.auc(fpr4, tpr4)
print("El AUC con SVM y PCA es = " + str(auc_SVM_pca))

In [ ]:
# Crear la figura y los subplots (1 fila, 2 columnas)
fig, axes = plt.subplots(1, 2, figsize=(12, 6))  # 1 fila, 2 columnas

# Gráfico 1: ROC Curve
axes[0].plot(fpr4, tpr4, lw=2, alpha=0.7, label='ROC Curve', color='b')
axes[0].plot(["No se fueron", "Se fueron"], ["No se fueron", "Se fueron"], linestyle='--', lw=1, color='r', label='Luck', alpha=.8)
axes[0].set_xlabel('Falsos Positivos Rate')
axes[0].set_ylabel('Verdaderos Positivos Rate')
axes[0].grid(False)
axes[0].legend(loc="lower right")
axes[0].set_title('ROC con SVM Y PCA ')

# Gráfico 2: Matriz de Confusión
cm4 = confusion_matrix(y_test, y_pred_pca_svm)

disp = ConfusionMatrixDisplay(confusion_matrix=cm4, display_labels=clf_SVM.classes_)
disp.plot(cmap='Blues', ax=axes[1])  # Usar el segundo eje para el gráfico
axes[1].set_title("Matriz de Confusión SVM y PCA")

plt.show()

In [ ]:
#Creamos un DF para guardar los resultados
resultados = pd.DataFrame(columns=['Modelo', 'Precision', 'Precision con PCA'])

In [ ]:
#agregamos los resultados de cada modelo al DF
resultados.loc[len(resultados)] = ['Regresion Logistica', accuracy, accuracy_pca]
resultados.loc[len(resultados)] = ['Random Forest', accuracy_rf, accuracy_pca_rf]
resultados.loc[len(resultados)] = ['SVM', accuracy_svm, accuracy_pca_svm]

In [ ]:
#mostramos los resultados
print(resultados)